In [ ]:
import pandas as pd

# Ler o CSV
df = pd.read_csv("bank_marketing/bank.csv", sep=";")

# eliminar duplicados
df.drop_duplicates(inplace=True)

# Mostrar número de linhas
print(f"Número de linhas: {len(df)}")

# Normalizar nomes das colunas
df.columns = df.columns.str.strip().str.lower()

# Contagem de 'unknown' em percentagem
percentagem_nulos = (df == "unknown").sum() * 100 / len(df)
print("\nPercentagem de 'unknown' por coluna:")
print(percentagem_nulos)

#modalizar unknown 
df['marital'] = df['marital'].replace('unknown', df['marital'].mode()[0])
df['education'] = df['education'].replace('unknown', df['education'].mode()[0])
df['housing'] = df['housing'].replace('unknown', df['housing'].mode()[0])
df['loan'] = df['loan'].replace('unknown', df['loan'].mode()[0])
df['job'] = df['job'].replace('unknown', df['job'].mode()[0])

#   converter meses para valores ordinais
month_mapping = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6,
                 "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}

df["month_ordinal"] = df["month"].map(month_mapping)

#one hot encoding para meses
df_month = pd.get_dummies(df["month"], prefix="month")
df = pd.concat([df, df_month], axis=1)
df.drop("month", axis=1, inplace=True)

#converter dias da semana para valores ordinais
df_day = pd.get_dummies(df["day_of_week"], prefix="day")
df = pd.concat([df, df_day], axis=1)
df.drop("day_of_week", axis=1, inplace=True)




#contagem de nonexists
contagem_nonexists = (df["poutcome"] == "nonexistent").mean() * 100
print(f"\nPercentagem de 'nonexistent' em 'poutcome': {contagem_nonexists:.2f}%")

#contagem de 0 em previous
contagem_0 = (df["duration"] == 0).mean() * 100
print(f"\nPercentagem de '0' em 'duration': {contagem_0:.2f}%")

#eliminar coluna duration
df = df.drop(columns=["duration"])


#contagem de valores 999 em pdays
contagem_999 = (df["pdays"] == 999).mean() * 100
print(f"\nPercentagem de '999' em 'pdays': {contagem_999:.2f}%")

#eliminar coluna pdays
df = df.drop(columns=["pdays"]) 

# Distribuição da variável alvo (em percentagem)
print("\nDistribuição da variável alvo (y):")   
print(df["y"].value_counts(normalize=True) * 100)

#separar variaveis independentes do alvo
X = df.drop(columns=["y"])
y = df["y"] 

#importar min e max scaler 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['age', 'campaign', 'previous']] = scaler.fit_transform(df[['age', 'campaign', 'previous']])

#padronizar colunas categoricas
df = pd.get_dummies(df, columns=['job', 'marital', 'education', 'default', 
                                 'housing', 'loan', 'contact', 'poutcome'], drop_first=True)

#verificar se existe valores nulos  
print("\nValores nulos por coluna:")
print(df.isnull().sum())




Número de linhas: 41176

Percentagem de 'unknown' por coluna:
age                0.000000
job                0.801438
marital            0.194288
education          4.201477
default           20.876239
housing            2.404313
loan               2.404313
contact            0.000000
month              0.000000
day_of_week        0.000000
duration           0.000000
campaign           0.000000
pdays              0.000000
previous           0.000000
poutcome           0.000000
emp.var.rate       0.000000
cons.price.idx     0.000000
cons.conf.idx      0.000000
euribor3m          0.000000
nr.employed        0.000000
y                  0.000000
dtype: float64

Percentagem de 'nonexistent' em 'poutcome': 86.34%

Percentagem de '0' em 'duration': 0.01%

Percentagem de '999' em 'pdays': 96.32%

Distribuição da variável alvo (y):
y
no     88.733728
yes    11.266272
Name: proportion, dtype: float64

Valores nulos por coluna:
age                              0
campaign                         0

In [38]:
#versão chatGPT
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# -----------------------------
# 1️⃣ Ler CSV e remover duplicados
# -----------------------------
df = pd.read_csv("bank_marketing/bank.csv", sep=";")
df.drop_duplicates(inplace=True)

# Normalizar nomes das colunas
df.columns = df.columns.str.strip().str.lower()

print(f"Número de linhas após duplicados: {len(df)}")

# -----------------------------
# 2️⃣ Verificação e tratamento de 'unknown'
# -----------------------------
# Percentagem de 'unknown' por coluna
percentagem_nulos = (df == "unknown").sum() * 100 / len(df)
print("\nPercentagem de 'unknown' por coluna:")
print(percentagem_nulos)

# Substituir 'unknown' pela moda das colunas categóricas
for col in ['marital', 'education', 'housing', 'loan', 'job']:
    df[col] = df[col].replace('unknown', df[col].mode()[0])

# -----------------------------
# 3️⃣ Transformação de variáveis categóricas
# -----------------------------

# Variáveis ordinal ou one-hot: mês e dia da semana
month_mapping = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6,
                 "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
df['month_ordinal'] = df['month'].map(month_mapping)

day_mapping = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5}
df['day_ordinal'] = df['day_of_week'].map(day_mapping)

# Outras variáveis categóricas → one-hot encoding
categorical_cols = ['job', 'marital', 'education', 'default', 
                    'housing', 'loan', 'contact', 'poutcome']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Remover colunas originais já processadas
df.drop(columns=['month', 'day_of_week'], inplace=True)

# -----------------------------
# 4️⃣ Tratar colunas irrelevantes
# -----------------------------
# Percentagem de 0 em duration
duration_zero_pct = (df['duration'] == 0).mean() * 100
print(f"\nPercentagem de '0' em 'duration': {duration_zero_pct:.2f}%")
df.drop(columns=['duration'], inplace=True)

# Percentagem de 999 em pdays
pdays_999_pct = (df['pdays'] == 999).mean() * 100
print(f"\nPercentagem de '999' em 'pdays': {pdays_999_pct:.2f}%")
df.drop(columns=['pdays'], inplace=True)

# -----------------------------
# 5️⃣ Variável alvo
# -----------------------------
df['y'] = df['y'].map({'no': 0, 'yes': 1})
print("\nDistribuição da variável alvo (y):")
print(df['y'].value_counts(normalize=True) * 100)

# -----------------------------
# 6️⃣ Normalizar colunas numéricas
# -----------------------------
scaler = MinMaxScaler()
numeric_cols = ['age', 'campaign', 'previous']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# -----------------------------
# 7️⃣ Verificação final
# -----------------------------
print("\nValores nulos por coluna:")
print(df.isnull().sum())

print("\nColunas finais do DataFrame:")
print(df.columns)



Número de linhas após duplicados: 41176

Percentagem de 'unknown' por coluna:
age                0.000000
job                0.801438
marital            0.194288
education          4.201477
default           20.876239
housing            2.404313
loan               2.404313
contact            0.000000
month              0.000000
day_of_week        0.000000
duration           0.000000
campaign           0.000000
pdays              0.000000
previous           0.000000
poutcome           0.000000
emp.var.rate       0.000000
cons.price.idx     0.000000
cons.conf.idx      0.000000
euribor3m          0.000000
nr.employed        0.000000
y                  0.000000
dtype: float64

Percentagem de '0' em 'duration': 0.01%

Percentagem de '999' em 'pdays': 96.32%

Distribuição da variável alvo (y):
y
0    88.733728
1    11.266272
Name: proportion, dtype: float64

Valores nulos por coluna:
age                              0
campaign                         0
previous                         0
emp.

In [ ]:



#



In [34]:
#Amostra exploratória estratificada
# df tem colunas: 'customer_id', 'date', 'y'
sample_frac = 0.05

def sample_group(x):
    n = max(1, int(round(len(x) * sample_frac)))  # pelo menos 1 por grupo
    return x.sample(n=n, random_state=42)

df_small = df.groupby('y', group_keys=False).apply(sample_group).reset_index(drop=True)


#Split temporal (treino/val/test por datas)
df['day_of_week'] = pd.to_datetime(df['day_of_week'])

# Define limites (ajusta as datas conforme os teus dados)
train_end = '2011-12-31'
val_end   = '2012-12-31'

train = df[df['day_of_week'] <= train_end].copy()
val   = df[(df['day_of_week'] > train_end) & (df['day_of_week'] <= val_end)].copy()
test  = df[df['day_of_week'] > val_end].copy()

print(len(train), len(val), len(test))

# oversampling
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_train = train.drop(columns=['y','customer_id','date'])
y_train = train['y']

# Pipeline exemplo
pipe = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('clf', LogisticRegression(max_iter=1000))
])

pipe.fit(X_train, y_train)
# GroupKFold por customer_id (para CV que evita vazamento entre clientes)
from sklearn.model_selection import GroupKFold, cross_val_score

X = train.drop(columns=['y','date'])
y = train['y']
groups = train['customer_id']

gkf = GroupKFold(n_splits=5)
for fold, (tr_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
    # aplica oversampling apenas em X_tr/y_tr se quiser


import numpy as np

def precision_at_k(y_true, y_score, k=0.05):
    # k como fracção do dataset (ex. 0.05 = top 5%)
    n = int(len(y_score) * k)
    idx = np.argsort(y_score)[::-1][:n]
    return y_true.iloc[idx].mean()

# após treinar o modelo e obter scores no test:
y_scores = model.predict_proba(X_test)[:,1]
print("Precision@5%:", precision_at_k(y_test.reset_index(drop=True), pd.Series(y_scores), k=0.05))





C:\Users\Aoazevedo\AppData\Local\Temp\ipykernel_22764\678263784.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby('y', group_keys=False).apply(sample_group).reset_index(drop=True)
C:\Users\Aoazevedo\AppData\Local\Temp\ipykernel_22764\678263784.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['day_of_week'] = pd.to_datetime(df['day_of_week'])


DateParseError: Unable to parse datetime string: mon, at position 0

In [22]:
#eliminar quem já falhou pagamentos
df.drop(df[df["default"] == "yes"].index, inplace=True)
print(len(df))

41185


In [ ]:
linhas_filtradas = df[((df["job"] == "unknown") | (df["marital"] == "unknown")) & (df["y"] == "yes")]
print(len(linhas_filtradas))
#eliminar quem tem job ou marital unknown e y yes
df = df[~(((df["job"] == "unknown") | (df["marital"] == "unknown")) & (df["y"] == "yes"))]
print(len(df))


linhas_filtradas = df[(df["previous"] == 0) ]
print(len(linhas_filtradas))

linhas_filtradas = df[(df["housing"] == "yes") & (df["loan"] == "yes") & (df["y"] == "yes")]
print(len(linhas_filtradas))
linhas_filtradas = df[(df["housing"] == "yes") &  (df["y"] == "yes")]
print(len(linhas_filtradas))
linhas_filtradas = df[ (df["loan"] == "yes") & (df["y"] == "yes")]
print(len(linhas_filtradas))
linhas_filtradas = df[ (df["marital"] == "divorced")  & (df["y"] == "yes")]
print(len(linhas_filtradas))
linhas_filtradas = df[ (df["marital"] == "single")  & (df["y"] == "yes")]
print(len(linhas_filtradas))  
linhas_filtradas = df[ (df["marital"] == "married")  & (df["y"] == "yes")]
print(len(linhas_filtradas))  







46
41139
35534
age                  0
job                271
marital             54
education         1453
default           8018
housing            851
loan               851
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64
404
2483
678
473
1605
2516


In [57]:
count_loan = df["loan"].value_counts()
print(count_loan) 

df_loan = df.drop(df[df["loan"] != "yes"].index, inplace=True)

n =len(df_loan)
 

df_loan.drop(df_loan[df_loan["default"] == "yes"].index, inplace=True)
nlen(df_loan)



loan
no         33947
yes         6248
unknown      990
Name: count, dtype: int64


TypeError: object of type 'NoneType' has no len()

In [ ]:
df = df.iloc[:n//3]
df.head()